# Universal estimator

Let $f(x|d_1,...,d_n)$ be a function, that for any fixed values of the parameters $d_i$, reduces to a PDF of x; $f$ thus is a family of functions, e.g., *log normal*.

$estimator(f, sample)$ is a function which learns the parameters $d_i$ of $f$ from a single input *sample* ($m$ observations drawn using $f$).

## Method
> 1. Generate synthetic *samples*.
> 2. Learn a DNN model from the synthetic data.
> 3. Predict the parameters $d_i$ on the input *sample*.

## Parameter range adjustment
The *estimator* is allowed to assume that the range of the parameter values is $(-\frac{1}{2}, \frac{1}{2})$.

For that we use *Range Adjustment*:
> Input:
> 1. A funation $f(x|d_i)$ as defined above
> 2. Bounds on the parameters $d_i$

> Output:
> - A function $g(x|t_i)$, which is the same as $f$, except that $t_i$ are in the range $(-\frac{1}{2}, \frac{1}{2})$, 
> and such that $t_i$ are “typically” around 0.

> Let: $h(d)$ be a mapping function from $range(d)$ to the range $(-\frac{1}{2}, \frac{1}{2})$, where $h$ is continuous within $range(d)$.
>
> $h^{-1}(t)$ is therefore a mapping function from the range $(-\frac{1}{2}, \frac{1}{2})$ to $range(d)$.
>
> Let: $g(x|t_i) = f(x|h^{-1}(t_i))$ where $h^{-1}(t_i)$ is a inverse of the *mapping* $h(d)$ of the bounds on the parameters $d_i$.

We use the *range adjustment* as follows:
> 1. Generate synthetic *samples* using $f(x|h^{-1}(t_i))$ where $t_i$ are drawn from $(-\frac{1}{2}, \frac{1}{2})$.
> 2. Learn a DNN model from the synthetic data.
> 3. Predict the parameters $d_i$ on the input *sample*.


### Range adjustment using $logit(x)$
The standard *logistic* function is defined as: $𝜎(x) = 1 / (1 + e^{-x})$ for $x ∈ (-∞, ∞)$.

<p style="text-align:center;"><img src="images/logistic.png" alt="logistic(x)" title="logistic(x), x ∈ (−∞,∞)" /></p>

The *logit* is the inverse of the *logistic* function.

The *logit* function is defined as: $logit(x) = 𝜎^{-1}(x) = ln( x / (1-x) )$ for $x ∈ (0,1)$.

<p style="text-align:center;"><img src="images/logit.png" alt="logit" title="logit(x), x ∈ (0,1)" /></p>

We can use the *logistic* function and its *logit* inverse as follows:
> Let: $h(d) = logit(d)$ for $d ∈ (0,1)$
>
> Than $h^{-1}(t) = 𝜎(t)$ for $t ∈ (-∞, ∞)$

> We define a function to return another function, i.e., an adjuster of a parameter from the range $(0, 1)$ 
>
> to the original range $(low, high)$ that a function takes.


In [1]:
import math
# from scipy.special import expit, logit

def expit(x):
    # expit(x) = 1 / (1 + e^(-x))
    # defined in (-INF, INF)
    return 1 / (1 + math.exp(-x))

def logit(p):
    # logit(p) = inv(expit) = ln(p/(1-p))
    # defined in (0, 1)
    if 0 == p: return -math.inf
    if 1 == p: return math.inf
    return math.log(p/(1-p))

def adjuster_logit(low, high):
    # adjuster is defined in (-INF, INF)
    # low: parameter lower bound
    # high: parameter higher bound
    # return: a function defined in (0,1) that maps it's parameter (x) to the original range (low,high).

    LOW = 0 if -math.inf == low else expit(low)
    HIGH = 1 if math.inf == high else expit(high)

    def adjust(x):
        # adjust is defined in (0, 1)
        if x < 0: return -math.inf
        if x > 1: return math.inf
        return logit(LOW + x * (HIGH - LOW))
    
    return adjust


### Range adjustment using $arctan(x)$
> <img src="images/tan.png" />
> <img src="images/arctan.png" />


In [2]:
def adjuster_arctan(low, high):
    # adjuster is defined in (-INF, INF)
    # low: parameter lower bound
    # high: parameter higher bound
    # return: a function defined in (0,1) that maps it's parameter (x) to the original range (low,high).

    LOW = -math.pi/2 if -math.inf == low else math.atan(low)
    HIGH = math.pi/2 if math.inf == high else math.atan(high)

    def adjust(x):
        # adjust is defined in (0, 1)
        if x < 0: return -math.inf
        if x > 1: return math.inf
        return math.tan(LOW + x * (HIGH - LOW))
    
    return adjust

### Test adjusters

In [3]:
print(f"adjuster_logit(2,10)(0): {adjuster_logit(low=-2, high=10)(0)}")
print(f"adjuster_logit(2,10)(1): {adjuster_logit(low=-2, high=10)(1)}")
print(f"adjuster_arctan(2,10)(0): {adjuster_arctan(low=-2, high=10)(0)}")
print(f"adjuster_arctan(2,10)(1): {adjuster_arctan(low=-2, high=10)(1)}")


adjuster_logit(2,10)(0): -2.0
adjuster_logit(2,10)(1): 10.00000000000097
adjuster_arctan(2,10)(0): -1.9999999999999996
adjuster_arctan(2,10)(1): 10.00000000000001


In [4]:
# adjust = adjuster_logit(low=0, high=10.0)
# adjust( np.random.uniform(0.0, 1.0, size=1)[0])

# Universal estimator

-  Apply 2D  range adjustment; for now, it will be independent parameters: should adjust to $(-\frac{1}{2}, \frac{1}{2})^2, (n=2)$.  
   In this case: we want the range to be an n-dimensional unit cube; this makes it easier to understand subranges: e.g., 0.1-0.2.  
   We also introduce some "prior" in a very convoluted way. A priori, the range is -∞ to +∞ and we know nothing about where the parameter is. After range adjustment, we shrink some regions by a large factor and others by a small factor. So, we favor the center of the range. We may even beat Cramer-Rao for this reason.

-  Select 5 or so longtail distributions (we may need to change the families later). Do range adjustment, and it should be sensible in the sense that it is approximately centered.

- Do range adjustment (orthogonal for now, i.e, independently for each original parameter).

- Draw a mesh on the unit square, say with a resolution of 0.1, this gives 81 points for each mesh. Maybe 1/11 resolution? Maybe include the last and first row and column. 

- For each point in the mesh, run learning experiment producing:
   - Several (say 10-100) learning experiments. without focusing for now.
   - Compute the average error of the 10-100 experiments. MSE rules
   - Compute the Cramer Rao bound (numerically if the analytical solution is not possible)
   - Divide the MSE by Cramer Rao.
  
- Draw the 3D manifold on the mesh.

### Expected results:

1. Hopefully, we are at most 2 times worse than CR. 
2. Hopefully, the CR is never singular, i.e. not zero nor infinity.


## Generate data

In [5]:
import math
import numpy as np

def generate_data(N, M, f, ranges, nbins=-1):
    """
    N:      number of samples
    M:      number of observations in each sample
    f:      the function to generate samples
    ranges: [dX2] array of parameter ranges (e.g. for 2 parmeters: [[0, 10], [0, math.inf]])
    """
    
    # output (generate samples)
    samples = np.zeros((N, M), dtype=float)
    
    ti_low = 0.0
    ti_high = 1.0
    
    # generate N parameter cubes in the range (ti_low, ti_high)
    n_params = ranges.shape[0]
    params = np.zeros((N, n_params), dtype=float)

    param_adjusters = [adjuster_logit(low=r[0], high=r[1]) for r in ranges]
        
    for i in range(N):
        for param_i in range(n_params):
            adjuster = param_adjusters[param_i]
            # ti is in (ti_low, ti_high)
            ti = np.random.uniform(ti_low, ti_high, size=1)[0]
            # di is in (di_low, di_high)
            params[i, param_i] = adjuster(ti)
    
    # repeat N times: draw a sample from distribution (M observations)
    for i in range(N):
        samples[i, :] = f(params[i], size=M)
    
    # create a histogram from each sample
    if nbins < 0:
        nbins = int(np.max(samples))+1
    
    histogram_matrix = np.apply_along_axis(
       lambda a: np.histogram(a, bins=nbins, range=(0, nbins), density=False)[0], 1, samples)

    return samples, params, histogram_matrix

###  Longtail distributions for testing

In [6]:
from scipy import stats
from scipy.stats import yulesimon
from scipy.stats import powerlaw

def yulesimon_sample(params, size):
    return yulesimon.rvs(alpha=params[0], loc=params[1], size=size)

def powerlaw_sample(params, size):
    return powerlaw.rvs(a=params[0], loc=params[1], size=size)

dist = {
    'yulesimon': {
        'sample': yulesimon_sample,
        'ranges': np.array([[2.0, 3.0], [0.0, math.inf]])
    },
    'powerlaw': {
        'sample': powerlaw_sample,
        'ranges': np.array([[0.0, 10.0], [0.0, math.inf]])
    }
}

### Imports

In [7]:
%run dnn.ipynb
from timeit import default_timer as timer

Tensorflow version 2.6.0


### Experiments

In [8]:
# import dnn library
   
def experiment(dist):
    
    N = 1000
    M = 256
    f = dist['sample']
    ranges = dist['ranges']
#     NUM_LEARNING_EXPERIMENTS = 10
    NUM_LEARNING_EXPERIMENTS = 1

    # create a mesh on the unit square, with a resolution of 0.1 (0.1,0.2,...,0.9)
    # The mesh is a cube of size 9^n_params (e.g., 9^2 for 2 params)
    n_params = ranges.shape[0]
    resolusion = np.arange(start=0.1, stop=1.0, step=0.1)
    resolusion = np.around(resolusion, decimals=1)
    n = len(resolusion)
    mesh = np.zeros(shape=[n]*n_params, dtype=float)

    # create range adjusters for each parameter range
    param_adjusters = [adjuster_logit(low=r[0], high=r[1]) for r in ranges]

    # - for each cell in the mesh create a sample using the corresponfing cell params.
    #   - adjust parameters in each cell using the appropriate adjuster for each parameter.
    mesh_samples = []
    mesh_params_ti = []
    mesh_params_di = []
    for idx in np.ndindex(mesh.shape):
        #lilo:ti = np.array(idx)/n + resolusion[0]
        ti = (np.array(idx)+1)/(n+1)
        di = np.zeros_like(ti)
        for i in range(n_params):
            di[i] = param_adjusters[i](ti[i])
        sample = f(di, size=M)
        mesh_samples.append(sample)
        mesh_params_ti.append(ti)
        mesh_params_di.append(di)
    
    # For each point in the mesh:
    # - Run learning experiment producing several (say 10-100) learning experiments.
    # - Compute the average error of the 10-100 experiments. MSE rules
    # - Compute the Cramer Rao bound (numerically if the analytical solution is not possible)
    # - Divide the MSE by Cramer Rao.
    
    num_samples = len(mesh_samples)
    
    mesh_errors = np.zeros(shape=(num_samples,), dtype=float)
    for i, sample in enumerate(mesh_samples):
        
        avg_mse = 0
        
        print(f'training mesh point - (ti: {mesh_params_ti[i]}), (di: {mesh_params_di[i]})')
        for j in range(NUM_LEARNING_EXPERIMENTS):
            
            start = timer()
            
            # generate training data
            samples_train, params_train, H_train = generate_data(N=N, M=M, f=f, ranges=ranges)
            
            print(f'sample ({i}/{num_samples}) trial ({j}/{NUM_LEARNING_EXPERIMENTS}) ...', end=' ')
            
            # fit DNN model to the training data           
            dnn_model, history = fit_dnn_model(X_train=H_train, y_train=params_train)

            # create test data (a histogram generated from the sample at mesh point)
            nbins = H_train.shape[1]
            H_test = np.histogram(sample, bins=nbins, range=(0, nbins), density=False)[0]
            H_test = H_test.reshape((1,-1))
            
            # predict sample parameters
            params_test = mesh_params_di[i].reshape((1,-1))
            y_pred, mse = predict(dnn_model, X_test=H_test, y_test=params_test)
            
            # average mse (accumulator)
            avg_mse += mse
            print(f'mse: {mse:.4f} ({int(timer()-start)} sec.)')
        
        # average mse for sample[i] predictions
        avg_mse /= NUM_LEARNING_EXPERIMENTS
        mesh_errors[i] = avg_mse
        print(f'done sample ({i}/{num_samples}) - (avg_mse: {avg_mse:.4f})')

experiment(dist['yulesimon'])


training mesh point - (ti: [0.1 0.1]), (di: [2.0702 0.2007])
sample (0/81) trial (0/1) ... mse: 0.0796 (11 sec.)
done sample (0/81) - (avg_mse: 0.0796)
training mesh point - (ti: [0.1    0.2111]), (di: [2.0702 0.4287])
sample (1/81) trial (0/1) ... mse: 0.0181 (5 sec.)
done sample (1/81) - (avg_mse: 0.0181)
training mesh point - (ti: [0.1    0.3222]), (di: [2.0702 0.6682])
sample (2/81) trial (0/1) ... mse: 0.0311 (8 sec.)
done sample (2/81) - (avg_mse: 0.0311)
training mesh point - (ti: [0.1    0.4333]), (di: [2.0702 0.928 ])
sample (3/81) trial (0/1) ... mse: 0.2799 (5 sec.)
done sample (3/81) - (avg_mse: 0.2799)
training mesh point - (ti: [0.1    0.5444]), (di: [2.0702 1.2209])
sample (4/81) trial (0/1) ... mse: 0.0325 (6 sec.)
done sample (4/81) - (avg_mse: 0.0325)
training mesh point - (ti: [0.1    0.6556]), (di: [2.0702 1.57  ])
sample (5/81) trial (0/1) ... mse: 0.1393 (5 sec.)
done sample (5/81) - (avg_mse: 0.1393)
training mesh point - (ti: [0.1    0.7667]), (di: [2.0702 2.024

KeyboardInterrupt: 